In [0]:
# Import neccessary libraries
from datetime import datetime
import pandas as pd
import numpy as np

!pip install metpy
from metpy.io import parse_metar_to_dataframe
import glob

In [0]:
 from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Read in the data
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2000/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2001/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2002/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2003/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2004/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2005/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2006/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2007/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2008/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2009/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2010/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2011/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2012/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2013/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2014/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2015/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2016/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2017/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2018/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2019/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2020/64010KMSN*.dat'

In [0]:

def parse_metar_file(file, wx_subset=True):
    """
    Parses METAR file from NCDC

    Input:
    --------
    file = Text file downloaded from NCDC

    wx_subset = Flag to determine whether or not to drop non-current weather obs (if True, only returns obs with observed weather)

    Output:
    --------
    df = Pandas dataframe filtered for times where current weather is not 'nan'
    """

    # Read in the file using Pandas
    df = pd.read_csv(file, header=None)

    # Pull the timestamp from the filename
    timestamp = datetime.strptime(file[-10:], '%Y%m.dat')

    # Iterrate over rows to parse METARS
    df_list = []
    for index, row in df.iterrows():
        try:
            df_list.append(parse_metar_to_dataframe(row.values[0][52:], year=timestamp.year, month=timestamp.month))
        except:
            print('Error with METAR: ', row.values[0][52:])
    #
    merged_df = pd.concat(df_list)

    # Drop datasets that do not include current weather
    merged_df = merged_df.dropna(subset=['current_wx1'])

    # Change the index to datetime
    merged_df.index = merged_df.date_time

    # Return the merged dataset sorted by datetime
    return merged_df.sort_index()




In [0]:
# Find all files in the data directory
files = sorted(glob.glob('*.dat'))

# Create a list to store the dataframes in
merged_datasets = []


In [0]:

# Loop through and parse the different datasets
for file in files:
    print(file[-10:])
    try:
        merged_datasets.append(parse_metar_file(file))
    except:
        print("Error with :", file)



In [0]:
# Return the cleaned dataset
#clean_df = (merged_datasets)
decoded_metar_master = pd.concat(merged_datasets)

## Save the file to memory
#clean_df.to_csv('decoded_metar_dataset.csv')
# Print out the resulting dataset
decoded_metar_master

In [0]:
# Create month column and add to the dataframe
time = pd.to_datetime(decoded_metar_master['date_time'])
decoded_metar_master['month'] = time.dt.month
decoded_metar_master

In [0]:
#decoded_metar_master = decoded_metar_master.dropna(axis=0, subset=['current_wx1','air_temperature'])

## Drop columns we don't plan on using or have too many NaNs
decoded_metar_master = decoded_metar_master.drop(['low_cloud_type',	'low_cloud_level',	'medium_cloud_type',	'medium_cloud_level',	'high_cloud_type',	'high_cloud_level',	'highest_cloud_type',	'highest_cloud_level',	'cloud_coverage', 'air_pressure_at_sea_level'], axis=1)

#decoded_metar_master = decoded_metar_master.dropna(axis=0)

In [0]:
#convert current_wx columns to list and also check how many rows we have 
current_wx1 = decoded_metar_master['current_wx1'].to_list()
print(len(current_wx1))
current_wx2 = decoded_metar_master['current_wx2'].to_list()
print(len(current_wx2))
current_wx3 = decoded_metar_master['current_wx3'].to_list()
print(len(current_wx3))

#Check to make sure we have some frozen AND liquid weather reported
print(current_wx1[0:50])

In [0]:
# List all of the current weather identifiers to identify non-precipitation identifiers to be removed
print(decoded_metar_master.current_wx1.unique())

In [0]:
# Create non-precip variables list to replace with NaNs
non_precip = ['HZ', 'BR', 'FG', '-FZDZ', 'BL', 'TS', 'VCTS', 'MIFG', 'FZFG', '-SNDZ', 'VCFG', '+TS', '-TS', 'FZDZ', '-TSDZ']

In [0]:
# Replace non-precip variables with NaNs
decoded_metar_master.replace(non_precip, np.nan, inplace=True)
### Then drop current_wx2 and current_wx3 because they hold a lot of NaNs
decoded_metar_master = decoded_metar_master.drop(['current_wx2',	'current_wx3'], axis=1)
### Then drop all other rows with NaNs
decoded_metar_master = decoded_metar_master.dropna(axis=0)
### Check how many rows we are left with after cleaning up the dataframe
current_wx1 = decoded_metar_master['current_wx1'].to_list()
print(len(current_wx1))

In [0]:
### List of frozen and liquid precip options
fzn_list = ['-SNRA','SNRA','+SNRA','-SN','SN','+SN', '-TSSN', 'TSSN', '+TSSN']
liq_list = ['-RA','RA','+RA','-TS','TS','+TS','-TSRA','TSRA','+TSRA','-FZRA','FZRA','+FZRA']

In [0]:
### Set up a list to hold binary values (1 for frozen, 0 for liquid)
binary_list = []

### Loop over current_wx observations and determin if any are frozen precip. If so, set binary variable to 1. If not, set binary variable to 0.
for i in range(len(current_wx1)):
    if (current_wx1[i] in fzn_list) or (current_wx2[i] in fzn_list) or (current_wx3[i] in fzn_list):
        binary_list.append(1)
    else:
        binary_list.append(0)

### Fill a new column called 'fzn_or_liq' with the binary values.
decoded_metar_master['fzn_or_liq'] = binary_list


In [0]:
# Double check remaining parameters are correct and all precipitation
decoded_metar_master.current_wx1.unique()

In [0]:
decoded_metar_master

In [0]:
# Save the final data to a csv file
decoded_metar_master.to_csv('/content/drive/My Drive/decoded_metar_FINAL.csv')